This notebook is based on Projected Gradient Descent (PGB) for 
epsilon = 0.03
alpha = 0.01
num_iterations = 10
and hence saving the images !

In [ ]:
import os
from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import ViTForImageClassification

import matplotlib.pyplot as plt

In [ ]:
# Hyperparameters
epsilon = 0.03
alpha = 0.01
num_iterations = 10

In [ ]:
input_base_path = "/workspace/CIFAKE"
output_base_path = "/workspace/CIFAKE/output"
os.makedirs(output_base_path, exist_ok=True)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def pgd_attack(model, images, labels, epsilon, alpha, num_iter):
    adv_images = images.clone().detach().requires_grad_(True).to(device)
    for _ in range(num_iter):
        outputs = model(adv_images)["logits"]
        loss = nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        adv_images = adv_images + alpha * adv_images.grad.sign()
        delta = torch.clamp(adv_images - images, min=-epsilon, max=epsilon)
        adv_images = (
            torch.clamp(images + delta, min=0, max=1).detach_().requires_grad_(True)
        )
    return adv_images

In [ ]:
def process_and_attack_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for label in ["FAKE", "REAL"]:
        label_input_path = os.path.join(input_folder, label)
        label_output_path = os.path.join(output_folder, label)
        os.makedirs(label_output_path, exist_ok=True)

        image_files = os.listdir(label_input_path)
        for image_name in tqdm(image_files, desc=f"Processing {label} Images"):
            image_path = os.path.join(label_input_path, image_name)
            try:
                # Load image and apply transformations
                image = Image.open(image_path).convert("RGB")
                image_tensor = transform(image).unsqueeze(0).to(device)
                label_tensor = torch.tensor([0]).to(device)  # Dummy label

                # Perform PGD attack
                adv_image_tensor = pgd_attack(
                    model, image_tensor, label_tensor, epsilon, alpha, num_iterations
                )
                adv_image_np = (
                    adv_image_tensor[0].cpu().detach().numpy().transpose(1, 2, 0)
                )
                adv_image_np = (adv_image_np * 255).astype("uint8")
                adv_image = Image.fromarray(adv_image_np)

                # Save adversarial image
                adv_image.save(os.path.join(label_output_path, f"adv_{image_name}"))
            except Exception as e:
                print(f"Error processing {image_name}: {e}")

In [ ]:
# Load a single image
image_path = "test.jpg"
image = Image.open(image_path).convert("RGB")
image_tensor = transform(image).unsqueeze(0).to(device)
dummy_label = torch.tensor([0]).to(device)

# Generate adversarial image
adv_image_tensor = pgd_attack(
    model, image_tensor, dummy_label, epsilon, alpha, num_iterations
)


# Convert tensors back to images for visualization
def tensor_to_image(tensor):
    tensor = tensor.squeeze().cpu().detach()
    tensor = (tensor * 0.5 + 0.5).clamp(0, 1)  # Unnormalize
    return transforms.ToPILImage()(tensor)


original_image = tensor_to_image(image_tensor)
adversarial_image = tensor_to_image(adv_image_tensor)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(original_image)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Adversarial Image")
plt.imshow(adversarial_image)
plt.axis("off")

plt.show()


In [ ]:
# Process each folder in test directory (FAKE and REAL)
for label_folder in os.listdir(input_base_path):
    label_path = os.path.join(input_base_path, label_folder)
    output_label_path = os.path.join(output_base_path, label_folder)
    os.makedirs(output_label_path, exist_ok=True)

    for image_name in tqdm(os.listdir(label_path), desc=f"Processing {label_folder}"):
        image_path = os.path.join(label_path, image_name)
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0).to(device)

        # Dummy label (can be adjusted based on your use case)
        dummy_label = torch.tensor([0]).to(device)

        # Generate adversarial image
        adv_image_tensor = pgd_attack(
            model, image_tensor, dummy_label, epsilon, alpha, num_iterations
        )

        # Save adversarial image
        adv_image = adv_image_tensor.squeeze().cpu().detach()
        adv_image = (adv_image * 0.5 + 0.5).clamp(0, 1)  # Unnormalize
        adv_image_pil = transforms.ToPILImage()(adv_image)

        adv_image_pil.save(os.path.join(output_label_path, f"adv_{image_name}"))

print(f"Adversarial images saved to {output_base_path}!")

In [ ]:
import shutil

folder_path = "/workspace/CIFAKE/output"
output_zip = "/workspace/CIFAKE/output.zip"

# Create a zip archive
shutil.make_archive(output_zip.replace(".zip", ""), "zip", folder_path)